In [1]:
import openai
import pandas as pd # data manipulation and analysis
import numpy as np # python lib for working with arrays, linear algebra
import seaborn as sns # data exploration and visualisation
import matplotlib.pyplot as plt # data visulisation and graphical charting
from sklearn.model_selection import train_test_split # create 2 subsets of the data (training and testing)
from sklearn.metrics import accuracy_score # accuracy of correctly classified among all samples.
from sklearn.metrics import classification_report # generating a human-readable text report
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import string 
import re # determine if a given text fits the given regular expression 

In [5]:
data_fake = pd.read_csv('Datasets/Fake.csv')
data_true = pd.read_csv('Datasets/True.csv')
data_fake.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [6]:
data_true.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


Add a label column to the data and concatenate the two datasets

In [7]:
data_fake['label'] = 0
data_true['label'] = 1
data = pd.concat([data_fake, data_true])

In [8]:
data_fake.shape, data_true.shape # returns the shape of an array which is a tuple of integers containing entries 

((23481, 5), (21417, 5))

data_fake contains 23481 rows and 5 columns
data_true contains 21417 rows and 5 columns

In [14]:
print(data_fake.index)
print(data_true.index)

RangeIndex(start=0, stop=23471, step=1)
RangeIndex(start=0, stop=21417, step=1)


Split the data into training and testing sets
- data: the dataset to split into training and testing sets
- test_size: the proportion of the dataset to use for testing (e.g. 0.2 for 20%)
- random_state: the seed used by the random number generator for reproducibility

In [18]:
X_train, X_test, y_train, y_test = train_test_split(
    data['text'],  # the feature (text) to be used for training and testing
    data['label'],  # the target (label) to be predicted for training and testing
    test_size=0.2,  # use 20% of the data for testing
    random_state=42  # use a fixed seed for reproducibility
)

Train the TF-IDF vectorizer

In [ ]:
vectorizer = TfidfVectorizer()
X_train_vect = vectorizer.fit_transform(X_train)

Prompts to generate a news article based on user input which asks for what the news should be about.

In [ ]:
about = input("What would you like news about?\n")
article = openai.Completion.create(
    engine="text-davinci-002",
    prompt= "Genrate a news article about the following" + about,
    max_tokens=2048,
    n=1,
    stop=None,
    temperature=0.7,
)